In [2]:
# general imports
import numpy as np
import pandas as pd
from urllib.request import urlopen
import json
import os
import plotly.express as px

# Loading EPA Data

In [62]:
air_data_21 = pd.read_csv("./local_datasets/air_quality/daily_summaries/daily_no2_42602_2021.csv")
air_data_20 = pd.read_csv("./local_datasets/air_quality/daily_summaries/daily_no2_42602_2020.csv")
air_data_19 = pd.read_csv("./local_datasets/air_quality/daily_summaries/daily_no2_42602_2019.csv")

## Augmenting EPA Data

The EPA daily summary has the fips code broken apart into it's components. For ease of use I'll be adding it to the df

In [25]:

def FIPS_function(row):
    state = str(row['State Code']).zfill(2)
    county = str(row['County Code']).zfill(3)
    return str(state + county)
# convert to NO2 ug/m^3 for reference
def no2_mass_by_vol(ppb):
    ugm3 = 1.88*ppb
    return ugm3

# plotting one day's avg 
def show_day_mean(df, date):
    fig = px.choropleth(df[df['Date Local']==date], geojson=counties, locations='fips', color='Arithmetic Mean',
                               color_continuous_scale="Plasma",
                               range_color=(0, 70), #max value for daily avg is ~60ppb
                               scope="usa",
                               labels={'Arithmetic Mean':'Arithmetic Mean (ppb)'}
                              )
    fig.update_layout(margin={"r":0,"t":0,"l":0,"b":1,'autoexpand':True })
    fig.update_layout(
        autosize=False,
        width=1200,
        height=900,
    )
    return fig


# plotting one day's max value
def show_day_max(df, date):
    fig = px.choropleth(df[df['Date Local']==date], geojson=counties, locations='fips', color='1st Max Value',
                               color_continuous_scale="Plasma",
                               range_color=(0, 70), #max value for daily avg is ~60ppb
                               scope="usa",
                               labels={'1st Max Value':'1st Max Value (ppb)'}
                              )
    fig.update_layout(margin={"r":0,"t":0,"l":0,"b":1,'autoexpand':True })
    fig.update_layout(
        autosize=False,
        width=1200,
        height=900,
    )
    return fig   
    

In [64]:
air_data_20['fips'].iloc[0]

'01073'

In [65]:
nair_data_19 = air_data_19.astype({'fips':pd.StringDtype()})
nair_data_21 = air_data_21.astype({'fips':pd.StringDtype()})
nair_data_20 = air_data_20.astype({'fips':pd.StringDtype()})

In [68]:
nair_data_20.to_csv("./data/air_quality/no2/daily_no2_2020_with_FIPS.csv",index=False)
nair_data_19.to_csv("./data/air_quality/no2/daily_no2_2019_with_FIPS.csv",index=False)
nair_data_21.to_csv("./data/air_quality/no2/daily_no2_2021_with_FIPS.csv",index=False)

## Testing the Dtype change

In [6]:
df = pd.read_csv("./data/air_quality/no2/daily_no2_2020_with_FIPS.csv", dtype={'fips':'string'})

In [5]:
al_cols=list(df.columns)
with open('air_data_columns.txt','w') as fout:
    for name in al_cols:
        fout.write(name+'\n')

In [9]:
# len(str(df['fips'].iloc[0]))
len(str(list(df['fips'].unique())[0]))

5

## Can we make a single dataset for these?

In [ ]:
len(air_data_20)

150k + rows is a lot, we might have to get clever with this stuff

## Setting up US county map

Documentation for Plotly's county map system available [here](https://plotly.com/python/choropleth-maps/)

In [ ]:
# Pulling a GeoJSON file to give the geometry information for us counties, See docs for more details

#might want to pickle 
with urlopen('https://raw.githubusercontent.com/plotly/datasets/master/geojson-counties-fips.json') as response:
    counties = json.load(response)
# feature.id is a FIPS code


In [ ]:
# tweaked the plotly example to show where the monitoring sites are
fig = px.choropleth(air_data_20, geojson=counties, locations='fips', color='POC',
                           color_continuous_scale="Viridis",
                           range_color=(0, 12),
                           scope="usa",
                           # labels={'POC':'location of sites'}
                          )
# fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.update_layout(
    autosize=False,
    width=900,
    height=1100,)
fig.show()